In [1]:
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gmaps
import requests
import json
from pprint import pprint
from api_keys import g_key

In [2]:
pull_data = "weather_data.csv"
weather_df = pd.read_csv(pull_data)
weather_df = pd.DataFrame(weather_df)

In [3]:
gmaps.configure(g_key)

In [4]:
locations = weather_df[["Latitude", "Longitude"]]
humidity = weather_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Ideal weather data frame

In [6]:
perfect_place = weather_df[["City", "Country", "Latitude", "Longitude", "Max Temp", "Wind Speed", "Cloud Cover"]]


perfect_place_table = perfect_place[(perfect_place["Max Temp"] < 80) & (perfect_place["Max Temp"] > 70) & (perfect_place["Wind Speed"] < 10) & (perfect_place["Cloud Cover"] == 0)]
perfect_place_table

,City,Country,Latitude,Longitude,Max Temp,Wind Speed,Cloud Cover
66,arraial do cabo,BR,-22.97,-42.03,70.92,6.53,0.0
69,zhangye,CN,38.93,100.45,75.90,5.14,0.0
88,darnah,LY,32.77,22.64,70.02,5.06,0.0
211,gerash,IR,27.67,54.14,75.20,4.50,0.0
224,kishtwar,IN,33.32,75.77,73.98,2.01,0.0
265,awjilah,LY,29.11,21.29,71.65,7.43,0.0
310,san lawrenz,MT,36.06,14.20,75.99,9.17,0.0
320,abu samrah,SY,35.30,37.18,71.71,8.10,0.0
322,carutapera,BR,-1.20,-46.02,77.47,6.42,0.0
412,gat,IL,31.61,34.76,77.00,2.24,0.0


In [7]:
perfect_place_table.isnull().any()

City           False
Country        False
Latitude       False
Longitude      False
Max Temp       False
Wind Speed     False
Cloud Cover    False
dtype: bool

In [8]:
few_perfect = perfect_place_table.iloc[0:5, 0:7]
few_perfect

,City,Country,Latitude,Longitude,Max Temp,Wind Speed,Cloud Cover
66,arraial do cabo,BR,-22.97,-42.03,70.92,6.53,0.0
69,zhangye,CN,38.93,100.45,75.90,5.14,0.0
88,darnah,LY,32.77,22.64,70.02,5.06,0.0
211,gerash,IR,27.67,54.14,75.20,4.50,0.0
224,kishtwar,IN,33.32,75.77,73.98,2.01,0.0


In [9]:
hotel_df = few_perfect.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Max Temp,Wind Speed,Cloud Cover,Hotel Name
0,arraial do cabo,BR,-22.97,-42.03,70.92,6.53,0.0,
1,zhangye,CN,38.93,100.45,75.90,5.14,0.0,
2,darnah,LY,32.77,22.64,70.02,5.06,0.0,
3,gerash,IR,27.67,54.14,75.20,4.50,0.0,
4,kishtwar,IN,33.32,75.77,73.98,2.01,0.0,


In [10]:
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)
    
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


{
    "html_attributions": [],
    "next_page_token": "CrQCIQEAAGyiW8yfJRb-AaWRiqKVg8qpjL-sqlip6slLVxSXMsuxeM_u9g4Q9NMw7TY6PdxkT-FolKo9qr6os7bDtd1ZaDQ-5_ExhjrHc4S-FC7yfiUYp2aU44_iPqN60tkN-_dj4FVFrrbBXqEW0C3wZOC8-g7ZaeaysgqfQmTGBRAqAqYDQ0Iud3TkTpLr1_FVaWri88rwvm7X9d687UlTMgrQiqwUAngyYSXNAhPM2EtuNLI7KHcV0Inwc3ctYBSPM9XveBbweruv7RhIVUrw0v3MB3c1ENDIRx3l7dujwzV4-zcRUOXcL_JuH4Bc5hEtg2EttepG-tXOsRENZewCc06UAOtpsFWYC7eMg1LBUm_ME7An0FxFjKA1O32s2GC8yb5JMHtCPtANxIcpBYe8SNeU4BMSEGmh057ccA-h4uSV1qVwoikaFBwF5zzDPfavhMHHOAp67c6xwZ2B",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -22.8621864,
                    "lng": -41.9865202
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.86085072010728,
                        "lng": -41.98521972010727
                    },
                    "southwest": {
                       

{
    "html_attributions": [],
    "next_page_token": "CrQCIQEAADhNtIs9byRm0PiGzvYx9WbYkHcZ2WIFXf3GcVbtvkjQ3aNK-XB56xV5bWw5Nr8ES2wLQgqI1lh2Jh5pj9glI6Ag721er8az8Zf8JRlSqiw49YVG7sJfpPnV6YlXfyzfZssPiRru5x-t-L_h3PgSGnbkhT7Nme-MXWiI0u43BuDvgVH9IOQdgZYoDBdJetP7knUER8bJtKhm7PODZWuklNBU2UQKBKxkAbJDZwS7olh1ED7pkBrxy4H-w1WiGQ2YrvTR7hM8fqZxMB74Kcp-Ny1N5GyKf2IgQz0h0RXqJhTNhq8HTQmbY9pbNbb1PnEbkSAJY3tWZKzlsPhVtoyUhJrxIUWFxqMBaCm2BD6e2gNlaCwt5gzKgnnoC-1SFYzMa8Gvv7FHGi67FjihLHNf_GYSEMceb6WuHOj0K0hSRZLdkoYaFPA0as7X4n9bNrnG9eBg3tF-sIUy",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 38.9588,
                    "lng": 100.43428
                },
                "viewport": {
                    "northeast": {
                        "lat": 38.96010652989273,
                        "lng": 100.4358456298927
                    },
                    "southwest": {
                        "lat": 

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.7671991,
                    "lng": 22.637898
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.76851117989271,
                        "lng": 22.63929917989272
                    },
                    "southwest": {
                        "lat": 32.76581152010727,
                        "lng": 22.63659952010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "\u0641\u0646\u062f\u0642 \u0627\u0644\u0641\u0631\u062f\u0648\u0633",
            "place_id": "ChIJMRfzTGkggBQRd46JwUA1GPQ",
            "plus_code": {
                "compound_code": "QJ8Q+V5 Derna, Libya",
                "global_code": "8G44QJ8Q+V5"
      

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 27.6774469,
                    "lng": 54.3377238
                },
                "viewport": {
                    "northeast": {
                        "lat": 27.67881757989272,
                        "lng": 54.33900072989272
                    },
                    "southwest": {
                        "lat": 27.67611792010728,
                        "lng": 54.33630107010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "name": "\u0645\u0627\u0645\u0648\u0631\u0633\u0631\u0627",
            "place_id": "ChIJU8wb44S7Vj4Rbcpi1ckN1ww",
            "plus_code": {
                "compound_code": "M8GQ+X3 Lar, Fars Province, Iran",
                "global_code": "7HVPM8GQ+X3"
            

{
    "html_attributions": [],
    "next_page_token": "CrQCIQEAAM7YJ7DeS63Y_ROcAI3_cns7PdxtbopV6JD4ECfhaOPtneBtfeN9H0xMOJmhQzutbJ6T6NQ5Ks2twtY8Qok66a-wkHwusMerG7QT8IO0x163xCdxD7nldnI4DyJu5EUcQpKOFmT7I2kT4dfgDtvWtuXJRMIPBES2nuzLD1cOIAoBM6bk0KjV2PSDqjhQ6rwZMRelbSxlS0KVE4jAd8bhAo2-fhMO_gPCFEB98d4TU-8igvLQ9a9xAIIE7_7hOAX5GS8VvU70eYYRzP8SWuRaS07TUTGTKaQvXnB8QTAm9tkovRQubkbKikJxQRaZJGZq9vPXDFAJO2tq2JVcp4v2QUQKUamousDJmobPeGqN2M6rLRcFROwAh8vmeRoxpJOmmdL9C87N7XkZo4ba-TrmTA8SEP_s53Cy4AkP3Fl2vM_4LrkaFKQCF80Jvm2JL7Vm4s5pZ5V2JqQk",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 33.0716071,
                    "lng": 75.33562309999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.07268247989273,
                        "lng": 75.33687352989273
                    },
                    "southwest": {
                    

In [11]:
hotel_df

,City,Country,Latitude,Longitude,Max Temp,Wind Speed,Cloud Cover,Hotel Name
0,arraial do cabo,BR,-22.97,-42.03,70.92,6.53,0.0,Paradiso Peró Praia Hotel
1,zhangye,CN,38.93,100.45,75.90,5.14,0.0,Holiday Inn Express Zhangye
2,darnah,LY,32.77,22.64,70.02,5.06,0.0,فندق الفردوس
3,gerash,IR,27.67,54.14,75.20,4.50,0.0,مامورسرا
4,kishtwar,IN,33.32,75.77,73.98,2.01,0.0,"Hotel Samson, Patnitop"


In [12]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [13]:
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))